In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
def get_labels(ds):
    if ds == 'crema' or ds == 'crema_no_aug':
        label_list = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
    elif ds == 'bser' or ds == 'bser_no_aug':
        label_list = ['angry', 'happy', 'neutral', 'sad', 'surprise']
    else:
        label_list = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    
    return label_list

In [13]:
with open('vggish/tess.pkl', 'rb') as f:
    df = pickle.load(f)

df[0]['info']
df[0]

{'info': ([2048], 0.2, 'adamax', 'tess', 1e-05, 'vggish-emb', False),
 'res': [0.21672704815864563,
  781.0,
  54.0,
  4998.0,
  61.0,
  0.9287410974502563,
  0.9353293180465698,
  0.9275534152984619,
  0.9932213425636292,
  0.9754472374916077],
 'f1_weighted': 0.9287061040717902,
 'accuracy': 0.9287410926365796,
 'cm': array([[ 99,   2,   2,   0,   0,   0,   0],
        [  1, 148,   0,   0,   3,   3,   4],
        [  0,   0,  75,   4,   0,   0,   1],
        [  2,   1,   1,  86,   0,   0,   8],
        [  1,   1,   0,   0, 119,   7,   0],
        [  0,   2,   0,   0,   4, 153,   0],
        [  2,   3,   0,   8,   0,   0, 102]]),
 'report': "              precision    recall  f1-score   support\n\n    b'angry'     0.9429    0.9612    0.9519       103\n  b'disgust'     0.9427    0.9308    0.9367       159\n     b'fear'     0.9615    0.9375    0.9494        80\n    b'happy'     0.8776    0.8776    0.8776        98\n  b'neutral'     0.9444    0.9297    0.9370       128\n      b'sad'     0

In [14]:
dses = ['tess', 'ravdess', 'savee', 'crema', 'subesco', 'bser', 'all_en', 'all_bn', 'all']
folders = ['vggish', 'yamnet']

res = []

for ds in dses:
    for folder in folders:
        with open(f'./{folder}/{ds}.pkl', 'rb') as f:
            data = pickle.load(f)


        for datum in data:

            cm = datum['cm']
            cm_n = cm / cm.astype(float).sum(axis=1)
            label_names = get_labels(ds)

            # ax= plt.subplots()
            # sns.heatmap(cm_n, annot=True, fmt='.2g', ax=ax)  #annot=True to annotate cells, ftm='g' to disable scientific notation

            # # labels, title and ticks
            # ax.set_xlabel('Predicted labels')
            # ax.set_ylabel('True labels')

            # model_name = 'VGGish' if folder == 'vggish' else 'YAMNet'

            # ax.set_title(f'{model_name} on {ds.upper()}')
            # ax.xaxis.set_ticklabels(label_names)
            # ax.yaxis.set_ticklabels(label_names)

            # plt.savefig(f'./{folder}/{ds}.png')

            size, dropout, opt, _, lr, _, _ = datum['info']
            res.append(
                {
                    'dataset': ds,
                    'model': folder,
                    'size': ', '.join([str(s) for s in size]),
                    'dropout': dropout if dropout != 0.0 else 'None',
                    'cr': datum['report'],
                    'opt': opt,
                    'lr': lr,
                    'accuracy': datum['accuracy'],
                    'f1_weighted': datum['f1_weighted'],
                    'cm': cm,
                    'label_names': label_names,
                }
            )

df = pd.DataFrame(res)

best_df = pd.DataFrame()
for ds in dses:
    for folder in folders:
        best = df[(df['dataset'] == ds) & (df['model'] == folder)].sort_values('f1_weighted', ascending=False).iloc[0]
        # best_df = best_df.append(best)
        best_df = pd.concat([best_df, best.to_frame().T])
best_df.to_csv('best.csv', index=False)
best_df

,dataset,model,size,dropout,cr,opt,lr,accuracy,f1_weighted,cm,label_names
17,tess,vggish,"1024, 512",0.2,precision recall f1-score ...,adam,0.0001,0.941805,0.941578,"[[99, 2, 2, 0, 0, 0, 0], [0, 154, 0, 0, 1, 2, ...","[angry, disgust, fear, happy, neutral, sad, su..."
35,tess,yamnet,2048,0.5,precision recall f1-score ...,adamax,0.001,0.787189,0.788118,"[[267, 17, 5, 22, 0, 0, 21], [15, 418, 9, 18, ...","[angry, disgust, fear, happy, neutral, sad, su..."
49,ravdess,vggish,"512, 256",0.2,precision recall f1-score ...,adamax,0.0001,0.505495,0.497979,"[[22, 5, 6, 7, 3, 1, 4], [5, 30, 3, 4, 5, 2, 8...","[angry, disgust, fear, happy, neutral, sad, su..."
66,ravdess,yamnet,2048,0.2,precision recall f1-score ...,adamax,0.00001,0.40469,0.39745,"[[79, 20, 21, 26, 6, 10, 11], [26, 65, 15, 29,...","[angry, disgust, fear, happy, neutral, sad, su..."
87,savee,vggish,"2048, 1024",None,precision recall f1-score ...,adamax,0.0001,0.460265,0.46345,"[[11, 4, 3, 6, 2, 0, 5], [5, 17, 2, 1, 7, 3, 1...","[angry, disgust, fear, happy, neutral, sad, su..."
121,savee,yamnet,1024,0.5,precision recall f1-score ...,adam,0.001,0.360316,0.363174,"[[28, 24, 8, 8, 15, 3, 18], [11, 58, 11, 4, 28...","[angry, disgust, fear, happy, neutral, sad, su..."
146,crema,vggish,1024,None,precision recall f1-score ...,adam,0.0001,0.400516,0.400127,"[[199, 30, 6, 87, 39, 2], [39, 181, 72, 72, 11...","[angry, disgust, fear, happy, neutral, sad]"
160,crema,yamnet,4096,None,precision recall f1-score ...,adamax,0.00001,0.349352,0.335883,"[[706, 122, 81, 141, 135, 44], [212, 443, 171,...","[angry, disgust, fear, happy, neutral, sad]"
171,subesco,vggish,512,None,precision recall f1-score ...,adam,0.001,0.541831,0.541309,"[[199, 59, 3, 24, 2, 1, 40], [48, 114, 13, 60,...","[angry, disgust, fear, happy, neutral, sad, su..."
192,subesco,yamnet,"4096, 2048",0.5,precision recall f1-score ...,adamax,0.00001,0.40023,0.390755,"[[579, 174, 68, 95, 45, 23, 148], [209, 231, 8...","[angry, disgust, fear, happy, neutral, sad, su..."


In [45]:
import re

macro_f1 = []

for i in range(len(best_df)):
    row = best_df.iloc[i]
    print(row['dataset'], row['model'])
    print('---------------------------------')
    # cr = row['cr']
    cr = row['cr'].split('\n')[-3].strip()
    cr = re.split(' +', cr)[-2]
    print(cr)
    macro_f1.append(float(cr))
    print('\n\n')

tess vggish
---------------------------------
0.9404



tess yamnet
---------------------------------
0.7836



ravdess vggish
---------------------------------
0.4745



ravdess yamnet
---------------------------------
0.3760



savee vggish
---------------------------------
0.4101



savee yamnet
---------------------------------
0.3112



crema vggish
---------------------------------
0.4071



crema yamnet
---------------------------------
0.3393



subesco vggish
---------------------------------
0.5391



subesco yamnet
---------------------------------
0.3905



bser vggish
---------------------------------
0.6690



bser yamnet
---------------------------------
0.4978



all_en vggish
---------------------------------
0.5197



all_en yamnet
---------------------------------
0.4204



all_bn vggish
---------------------------------
0.5250



all_bn yamnet
---------------------------------
0.3904



all vggish
---------------------------------
0.5088



all yamnet
--------------

In [61]:
best_df['macro_f1'] = macro_f1
best_df

,dataset,model,size,dropout,cr,opt,lr,accuracy,f1_weighted,cm,label_names,macro_f1
17,tess,vggish,"1024, 512",0.2,precision recall f1-score ...,adam,0.0001,0.941805,0.941578,"[[99, 2, 2, 0, 0, 0, 0], [0, 154, 0, 0, 1, 2, ...","[angry, disgust, fear, happy, neutral, sad, su...",0.9404
35,tess,yamnet,2048,0.5,precision recall f1-score ...,adamax,0.001,0.787189,0.788118,"[[267, 17, 5, 22, 0, 0, 21], [15, 418, 9, 18, ...","[angry, disgust, fear, happy, neutral, sad, su...",0.7836
49,ravdess,vggish,"512, 256",0.2,precision recall f1-score ...,adamax,0.0001,0.505495,0.497979,"[[22, 5, 6, 7, 3, 1, 4], [5, 30, 3, 4, 5, 2, 8...","[angry, disgust, fear, happy, neutral, sad, su...",0.4745
66,ravdess,yamnet,2048,0.2,precision recall f1-score ...,adamax,0.00001,0.40469,0.39745,"[[79, 20, 21, 26, 6, 10, 11], [26, 65, 15, 29,...","[angry, disgust, fear, happy, neutral, sad, su...",0.3760
87,savee,vggish,"2048, 1024",None,precision recall f1-score ...,adamax,0.0001,0.460265,0.46345,"[[11, 4, 3, 6, 2, 0, 5], [5, 17, 2, 1, 7, 3, 1...","[angry, disgust, fear, happy, neutral, sad, su...",0.4101
121,savee,yamnet,1024,0.5,precision recall f1-score ...,adam,0.001,0.360316,0.363174,"[[28, 24, 8, 8, 15, 3, 18], [11, 58, 11, 4, 28...","[angry, disgust, fear, happy, neutral, sad, su...",0.3112
146,crema,vggish,1024,None,precision recall f1-score ...,adam,0.0001,0.400516,0.400127,"[[199, 30, 6, 87, 39, 2], [39, 181, 72, 72, 11...","[angry, disgust, fear, happy, neutral, sad]",0.4071
160,crema,yamnet,4096,None,precision recall f1-score ...,adamax,0.00001,0.349352,0.335883,"[[706, 122, 81, 141, 135, 44], [212, 443, 171,...","[angry, disgust, fear, happy, neutral, sad]",0.3393
171,subesco,vggish,512,None,precision recall f1-score ...,adam,0.001,0.541831,0.541309,"[[199, 59, 3, 24, 2, 1, 40], [48, 114, 13, 60,...","[angry, disgust, fear, happy, neutral, sad, su...",0.5391
192,subesco,yamnet,"4096, 2048",0.5,precision recall f1-score ...,adamax,0.00001,0.40023,0.390755,"[[579, 174, 68, 95, 45, 23, 148], [209, 231, 8...","[angry, disgust, fear, happy, neutral, sad, su...",0.3905


In [60]:
vggish_res = best_df.drop(best_df[best_df['model'] == 'yamnet'].index)
vggish_res = vggish_res.drop(columns=['model', 'size', 'dropout', 'opt', 'lr', 'cm', 'label_names', 'cr'])
vggish_res = vggish_res.reset_index(drop=True)
vggish_res['dataset'] = vggish_res['dataset'].apply(lambda x: x.replace('_', ' '))
vggish_res['dataset'] = vggish_res['dataset'].str.upper()
vggish_res = vggish_res.rename(columns={
    'accuracy': '\\textbf{Accuracy}', 
    'f1_weighted': '\\textbf{F1 Weighted}', 
    'macro_f1': '\\textbf{Macro F1}', 
    'dataset': '\\textbf{Dataset}'})
vggish_res = vggish_res.applymap(lambda x: str(round(x*100, 2)) if isinstance(x, float) else x)


vggish_res.style.hide(axis='index').to_latex(
    './vggish/tables/vggish_res.tex',
    position='h',
    position_float='centering',
    hrules=True,
    caption='VGGish Results',
    label='tab:vggish_res',
    column_format='{|c|c|c|c|}')

with open('./vggish/tables/vggish_res.tex', 'r') as f:
    lines = f.read()
with open('./vggish/tables/vggish_res.tex', 'w') as f:
    lines = lines.replace('\\toprule', '\\hline')
    lines = lines.replace('\\midrule', '\\hline')
    lines = lines.replace('\\bottomrule', '\\hline')
    f.write(lines)

vggish_res

,\textbf{Dataset},\textbf{Accuracy},\textbf{F1 Weighted},\textbf{Macro F1}
0,TESS,94.18,94.16,94.04
1,RAVDESS,50.55,49.8,47.45
2,SAVEE,46.03,46.35,41.01
3,CREMA,40.05,40.01,40.71
4,SUBESCO,54.18,54.13,53.91
5,BSER,65.35,65.37,66.9
6,ALL EN,50.24,50.28,51.97
7,ALL BN,52.78,52.73,52.5
8,ALL,50.87,50.61,50.88


In [62]:
# YAMNet

vggish_res = best_df.drop(best_df[best_df['model'] == 'vggish'].index)
vggish_res = vggish_res.drop(columns=['model', 'size', 'dropout', 'opt', 'lr', 'cm', 'label_names', 'cr'])
vggish_res = vggish_res.reset_index(drop=True)
vggish_res['dataset'] = vggish_res['dataset'].apply(lambda x: x.replace('_', ' '))
vggish_res['dataset'] = vggish_res['dataset'].str.upper()
vggish_res = vggish_res.rename(columns={
    'accuracy': '\\textbf{Accuracy}', 
    'f1_weighted': '\\textbf{F1 Weighted}', 
    'macro_f1': '\\textbf{Macro F1}', 
    'dataset': '\\textbf{Dataset}'})
vggish_res = vggish_res.applymap(lambda x: str(round(x*100, 2)) if isinstance(x, float) else x)


vggish_res.style.hide(axis='index').to_latex(
    './yamnet/tables/yamnet_res.tex',
    position='h',
    position_float='centering',
    hrules=True,
    caption='YAMNet Results',
    label='tab:yamnet_res',
    column_format='{|c|c|c|c|}')

with open('./yamnet/tables/yamnet_res.tex', 'r') as f:
    lines = f.read()
with open('./yamnet/tables/yamnet_res.tex', 'w') as f:
    lines = lines.replace('\\toprule', '\\hline')
    lines = lines.replace('\\midrule', '\\hline')
    lines = lines.replace('\\bottomrule', '\\hline')
    f.write(lines)

vggish_res

,\textbf{Dataset},\textbf{Accuracy},\textbf{F1 Weighted},\textbf{Macro F1}
0,TESS,78.72,78.81,78.36
1,RAVDESS,40.47,39.74,37.6
2,SAVEE,36.03,36.32,31.12
3,CREMA,34.94,33.59,33.93
4,SUBESCO,40.02,39.08,39.05
5,BSER,48.86,48.48,49.78
6,ALL EN,41.2,40.83,42.04
7,ALL BN,39.92,38.96,39.04
8,ALL,40.27,39.63,39.68


In [76]:
# VGGish best parameters

model = 'yamnet'
filename = f'{model}_best_params'
path = f'./{model}/tables/{filename}.tex'

df_hp = best_df[best_df['model'] == model]
df_hp = df_hp.reset_index(drop=True)
df_hp = df_hp.drop(columns=['model', 'accuracy', 'f1_weighted', 'cm', 'label_names', 'cr', 'macro_f1'])
df_hp['dataset'] = df_hp['dataset'].apply(lambda x: x.upper().replace('_', ' '))
df_hp['opt'] = df_hp['opt'].apply(lambda x: x.capitalize())
df_hp['dropout'] = df_hp['dropout'].apply(lambda x: 0.0 if x == 'None' else float(x))
df_hp['dropout'] = df_hp['dropout'].apply(lambda x: str(x*100))

df_hp = df_hp.rename(columns = {
    'dataset': '\\textbf{Dataset}',
    'size': '\\textbf{Hidden Layer Size}',
    'dropout': '\\textbf{Dropout Rate}',
    'opt': '\\textbf{Optimizer}',
    'lr': '\\textbf{Init. Learning Rate}'
})

caption = 'VGGish' if model == 'vggish' else 'YAMNet'

df_hp.style.hide(axis='index').to_latex(
    path,
    position='h',
    position_float='centering',
    hrules=True,
    caption=f'{caption} Best Parameters',
    label=f'tab:{model}_bp',
    column_format='{|c|c|c|c|c|}')

with open(path, 'r') as f:
    lines = f.read()
with open(path, 'w') as f:
    lines = lines.replace('\\toprule', '\\hline')
    lines = lines.replace('\\midrule', '\\hline')
    lines = lines.replace('\\bottomrule', '\\hline')
    f.write(lines)

df_hp

,\textbf{Dataset},\textbf{Hidden Layer Size},\textbf{Dropout Rate},\textbf{Optimizer},\textbf{Init. Learning Rate}
0,TESS,2048,50.0,Adamax,0.001
1,RAVDESS,2048,20.0,Adamax,0.00001
2,SAVEE,1024,50.0,Adam,0.001
3,CREMA,4096,0.0,Adamax,0.00001
4,SUBESCO,"4096, 2048",50.0,Adamax,0.00001
5,BSER,4096,0.0,Adamax,0.00001
6,ALL EN,4096,0.0,Adamax,0.0001
7,ALL BN,"4096, 2048",20.0,Adamax,0.0001
8,ALL,"4096, 2048, 1024",20.0,Adamax,0.001


In [ ]:
for idx, row in best_df.iterrows():
    cm = row['cm']
    cm_n = cm / cm.astype(float).sum(axis=1)
    label_names = row['label_names']

    # rcParams['figure.figsize'] = 40, 12

    plt.tight_layout()
    ax= plt.subplot()
    sns.heatmap(cm_n, annot=True, fmt='.2g', ax=ax)  #annot=True to annotate cells, ftm='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')

    model_name = 'VGGish' if row['model'] == 'vggish' else 'YAMNet'

    ax.set_title(f'{model_name} on {row["dataset"].upper()}')
    ax.xaxis.set_ticklabels(label_names)
    ax.yaxis.set_ticklabels(label_names)

    plt.savefig(f'./{row["model"]}/cm/{row["dataset"]}.png', dpi=150)